In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# KNeighborsRegressor

## Préparation des données V1 - 2023/11/15

Stratégie :    
On supprime la colonne date et la colonne way. On remplace les NAN par des 0

- Import des données

In [46]:
path = "../../data"
x_data = pd.read_csv(path+'/Xtrain_hgcGIrA.csv', sep=',') # features
y_data = pd.read_csv(path+'/Ytrain_yL5OjS4.csv', sep=',', usecols=[1]) # occupancy rate

- Mise en forme des données

In [47]:
x_data = x_data.drop("date", axis=1) # suppression de la colonne date
x_data = x_data.drop("way", axis=1) # suppression de la colonne way

#on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype('category')
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)


x_data["hour"] = x_data["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
) # on remplace les Nan de hour par des 0 et les heures par des entiers

#on remplace les Nan des pxqx par 0
x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]].fillna(0)

- Création d'un dataset de test et d'apprentissage

In [48]:
y_data = y_data.iloc[x_data.index]
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(
    x_data, y_data, test_size=0.2
)

## KNeighborsRegressor - 2023/11/15 - Préparation des données V1

- Import de KNeighborsRegressor

In [49]:
from sklearn.neighbors import KNeighborsRegressor

- Test sans jouer sur les paramètres

In [50]:
KN = KNeighborsRegressor()
KN.fit(x_data_train, y_data_train)

print("Score train", KN.score(x_data_train, y_data_train))
print("Score test", KN.score(x_data_test, y_data_test))

Score train 0.9799021146468689
Score test 0.9666271239763211


Le score est plutôt bon. On va maintenant essayer d'optimiser les paramètres

- Choix des paramètres

In [53]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from time import time

start = time()

my_kfold = KFold(n_splits=10, shuffle=True, random_state=0)

n_neighbors = [x for x in range(1,11)]
weights = ['uniform', 'distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
p = np.linspace(1,2.5,10)

tuned_parameters = {'n_neighbors': n_neighbors, 'weights' : weights, 'algorithm' : algorithm, 'p':p}  

KN_params = GridSearchCV(KNeighborsRegressor(),
                      tuned_parameters,
                      cv=my_kfold,
                      n_jobs=-1)

KN_params.fit(x_data_train, y_data_train)
print(KN_params.score(x_data_test, y_data_test), KN_params.best_params_)
print(str(time() - start)+ " sec")

0.9689247627603355 {'algorithm': 'auto', 'n_neighbors': 4, 'p': 1.0, 'weights': 'distance'}
1314.6770651340485 sec


Premier test avec 
- n_neighbors = [2, 5, 10, 25, 50, 100, 200]
- weights = ['uniform', 'distance']
- algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
- p = [1, 2, 5, 10, 25, 50, 100]

On obtient : 

0.9686036274176808 {'algorithm': 'auto', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}    
620.6240813732147 sec


Deuxième test avec 
- n_neighbors = [x for x in range(1,11)]
- weights = ['uniform', 'distance']
- algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
- p = np.linspace(1,5,10)

On obtient : 

0.9689247627603355 {'algorithm': 'auto', 'n_neighbors': 4, 'p': 1.0, 'weights': 'distance'}    
1487.465755224228 sec


On va juste préciser le paramètre p.

Troisième test avec 
- n_neighbors = [x for x in range(1,11)]
- weights = ['uniform', 'distance']
- algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
- p = np.linspace(1,2.5,10)

On obtient encore : 

0.9689247627603355 {'algorithm': 'auto', 'n_neighbors': 4, 'p': 1.0, 'weights': 'distance'}    
1314.6770651340485 sec

On se fixe sur les paramètres suivants :
- n_neighbors = 4
- weights = distance
- algorithm = auto (paramètre par défaut)
- p = 1.0